## Import libraries

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn,optim

## Download Dataset

In [ ]:
train_set = datasets.MNIST('/content',train=True,transform=transforms.ToTensor(),download=True)
test_set = datasets.MNIST('/content',train=False,transform=transforms.ToTensor(),download=True)
trainloader = torch.utils.data.DataLoader(train_set,batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(test_set,batch_size=64,shuffle=True)

## Plotting some data

In [ ]:
dataiter = iter(trainloader)
images,labels = dataiter.next()
print(images.shape)
print(labels.shape)
figure = plt.figure()
num_of_images = 60
for index in range(1,num_of_images+1):
    plt.subplot(6,10,index)
    plt.axis('off')
    plt.imshow(images[index].numpy().squeeze(),cmap='gray_r')

## Build the neural network

In [ ]:
input_size = 784
hidden_sizes = [128,64]
output_size = 10

model = nn.Sequential(
    nn.Linear(input_size,hidden_sizes[0]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[0],hidden_sizes[1]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[1],output_size),
    nn.LogSoftmax(dim=1)
)

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.003,momentum=0.9)
time0 = time()
epochs = 15
for e in range(epochs):
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1) # flatten the image
        optimizer.zero_grad() # training pass
        output = model(images)
        loss = criterion(output,labels)
        loss.backward() # backpropagation
        optimizer.step() # optimizes its weights here
        
print(f"\nTraining time in minutes = {(time()-time0)/60}")

## Testing

In [ ]:
correct_outputs = 0
total_outputs = 0
for images,labels in testloader:
    for i in range(len(labels)):
        img = images[i].view(1,784)
        with torch.no_grad():
            logps = model(img)
        
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label= probab.index(max(probab))
        true_label = labels.numpy()[i]
        if true_label == pred_label:
            correct_outputs += 1
        total_outputs += 1
print(f"\nModel Accuracy : {correct_outputs / total_outputs}")

## Save the Model

In [ ]:
torch.save(model,'./mnist_model.pt')